## Import and Clean Dataset

For your convenience, the data can be found here again: https://cocl.us/concrete_data. To recap, the predictors in the data of concrete strength include:

* Cement
* Blast Furnace Slag
* Fly Ash
* Water
* Superplasticizer
* Coarse Aggregate
* Fine Aggregate

Start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

Read the dataset into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Check The Number Of Data Points

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split Data Into Predictors And Target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

## Import Keras

#### Import The Keras Library

In [11]:
import keras

Using TensorFlow backend.


As you can see, the TensorFlow backend was used to install the Keras library.

Import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has three hidden layers each with 10 neurons and ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Import scikit-learn in order to randomly split the data into a training and test sets

In [14]:
from sklearn.model_selection import train_test_split

Splitting the data into a training and test sets by holding 30% of the data for testing

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

## Train and Test the Network

Call the function now to create our model.

In [16]:
# build the model
model = regression_model()

Next, we will train the model for 50 epochs.


In [17]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Epoch 1/50
 - 0s - loss: 1573.2987
Epoch 2/50
 - 0s - loss: 1549.4359
Epoch 3/50
 - 0s - loss: 1513.8566
Epoch 4/50
 - 0s - loss: 1460.6395
Epoch 5/50
 - 0s - loss: 1379.3666
Epoch 6/50
 - 0s - loss: 1254.3307
Epoch 7/50
 - 0s - loss: 1076.9426
Epoch 8/50
 - 0s - loss: 862.1968
Epoch 9/50
 - 0s - loss: 644.7585
Epoch 10/50
 - 0s - loss: 474.9312
Epoch 11/50
 - 0s - loss: 366.7813
Epoch 12/50
 - 0s - loss: 309.0300
Epoch 13/50
 - 0s - loss: 275.0583
Epoch 14/50
 - 0s - loss: 247.6231
Epoch 15/50
 - 0s - loss: 227.6641
Epoch 16/50
 - 0s - loss: 212.4540
Epoch 17/50
 - 0s - loss: 200.7044
Epoch 18/50
 - 0s - loss: 191.7499
Epoch 19/50
 - 0s - loss: 185.7954
Epoch 20/50
 - 0s - loss: 180.1541
Epoch 21/50
 - 0s - loss: 175.4858
Epoch 22/50
 - 0s - loss: 171.6844
Epoch 23/50
 - 0s - loss: 168.3533
Epoch 24/50
 - 0s - loss: 164.4677
Epoch 25/50
 - 0s - loss: 161.4673
Epoch 26/50
 - 0s - loss: 158.3926
Epoch 27/50
 - 0s - loss: 155.5889
Epoch 28/50
 - 0s - loss: 152.5515
Epoch 29/50
 - 0s - lo

Next we need to evaluate the model on the test data.

In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 149us/step


95.54871378284442

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Import the mean_squared_error function from Scikit-learn.

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

95.54871593213757 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [21]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 57.38335714988338
MSE 2: 59.97239396178607
MSE 3: 43.69251469732488
MSE 4: 38.041453401633454
MSE 5: 34.175898505646046
MSE 6: 33.051592595368916
MSE 7: 35.06026382199383
MSE 8: 26.97654236481799
MSE 9: 27.834298180144966
MSE 10: 29.55718881181143
MSE 11: 24.85766007136373
MSE 12: 22.528280042135986
MSE 13: 27.98204521376724
MSE 14: 29.311979682700148
MSE 15: 24.389464578968035
MSE 16: 20.08626358871707
MSE 17: 22.96993147285239
MSE 18: 24.617873577624078
MSE 19: 22.42308620959038
MSE 20: 24.866901305115338
MSE 21: 24.367385833394565
MSE 22: 22.59671462469502
MSE 23: 20.916128640035982
MSE 24: 22.531630377167637
MSE 25: 25.59353800653254
MSE 26: 23.117390999902028
MSE 27: 19.99895450135265
MSE 28: 21.565083031515474
MSE 29: 22.917903622377267
MSE 30: 18.411303640569297
MSE 31: 18.546056315351073
MSE 32: 20.094128642653185
MSE 33: 18.42688812329931
MSE 34: 20.16441692117734
MSE 35: 20.85664756552687
MSE 36: 23.97086848410202
MSE 37: 16.11257454338197
MSE 38: 22.210580029533904
MS